### Loading Data
Dan Constantini, Tom Hayat et Alexandre Attia
This script loads 11 data class from imagenet and sort the files according to their name

In [1]:
require 'torch'
require 'xlua'
require 'image'
require 'nn'
require 'optim'


--classes={'bridge', 'building', 'city', 'eiffel_tower','elephant', 'landscape', 'lion', 'monkey', 'people', 'tower', 'water'}
total = 0
for dir in paths.iterdirs("./dataset/train/") do
    files = {}
    for file in paths.files(paths.concat('./dataset/train/', dir)) do
       if file:find('JPEG' .. '$') then
          table.insert(files, 1)
       end
    end
   
    if #files == 0 then
       error('given directory doesnt contain any files of type: ')
    end

    total = total + #files
    collectgarbage()
end

print('total', total)

imagesAll = torch.Tensor(total,3,64,64) 
labelsAll = torch.Tensor(total)
compteur = 0
NumberOfImages = {0}
for dir in paths.iterdirs("./dataset/train/") do
    compteur = compteur +1
    files = {}
    for file in paths.files(paths.concat('./dataset/train/', dir)) do
       if file:find('JPEG' .. '$') then
          table.insert(files, paths.concat(paths.concat('./dataset/train/',dir), file))
       end
    end
   
    if #files == 0 then
       error('given directory doesnt contain any files of type: ')
    end

    lastElement = NumberOfImages[#NumberOfImages]
    table.insert(NumberOfImages, #files + lastElement)
    for i=1,(#files) do
        temp = image.load(files[i])
        temp2 = torch.Tensor(3,64,64) 
        if (temp:size()[1]==1) then
            temp2[{{1},{},{}}] = temp
            temp2[{{2},{},{}}] = temp
            temp2[{{3},{},{}}] = temp
        else 
            temp2 = temp
        end
        imagesAll[i+lastElement] = temp2 
        labelsAll[i+lastElement] = compteur
    end
    collectgarbage()
end


-- Nombre d'images courleurs: 1683 
trainData = {
    data = torch.Tensor(total, 3, 64, 64),
    labels = torch.Tensor(total),
    size = function() return total end,

}
--[[create test set:
testData = {
      data = torch.Tensor(tesize, 1, 32, 32),
      labels = torch.Tensor(tesize),
      size = function() return tesize end
   }
]]--

for i=1,total do
   trainData.data[i] = imagesAll[i]
   trainData.labels[i] = labelsAll[i]
end
--for i=trsize+1,tesize+trsize do
   --testData.data[i-trsize] = imagesAll[labelsShuffle[i]][1]:clone()
   --testData.labels[i-trsize] = labelsAll[labelsShuffle[i]]
--end

total	1718	


In [2]:
trainData

{
  data : DoubleTensor - size: 1718x3x64x64
  size : function: 0x0eb1ee58
  labels : DoubleTensor - size: 1718
}


In [3]:
--for i=1,total do 
    --itorch.image(imagesAll[i])
    --print(labelsAll[i])
--end

<b>Problème :</b> à partir de l'image 288, les images sont entierement noirs et le label est nul. Je pense qu'il y a un probleme avec le `if (temp:size()[1]==3) then` ! En fait c'est avant je crois, quand on print le nombre de files ils en mettent que 190 .. --> J'ai compris on a un probleme dans la boucle, en fait, elle repart de 0 à chaque fois pour imageAll, donc à chaque fois qu'on parcourt un repertory d'images on réécrit sur les images anciennes (la liste `files` est temporaire. Le nombres d'images dans imagesAll est donc 288 car le nombre maximal d'images des dossiers du dataset est 288. 

``for i=1,(#files) do
        print('dossier : ', compteur)
        print(#files)
        temp = image.load(files[i])
        --print(files[i])
        if (temp:size()[1]==3) then
            imagesAll[i] = image.load(files[i]) 
            labelsAll[i] = compteur
        end
    end
``

Est ce que ce code là on le sort pas de la grosse boucle ?

## Preprocessing Data

In [4]:
-- Convert all images to YUV
collectgarbage()
print '==> preprocessing data: colorspace RGB -> YUV'
for i = 1,total do
   trainData.data[i] = image.rgb2yuv(trainData.data[i])
end
--for i = 1,325 do
  -- testData.data[i] = image.rgb2yuv(testData.data[i])
--end
-- Name channels for convenience
channels = {'y','u','v'}

-- Normalize each channel, and store mean/std
-- per channel. These values are important, as they are part of
-- the trainable parameters. At test time, test data will be normalized
-- using these values.
print '==> preprocessing data: normalize each feature (channel) globally'
mean = {}
std = {}
for i,channel in ipairs(channels) do
   -- normalize each channel globally:
   mean[i] = trainData.data[{ {},i,{},{} }]:mean()
   std[i] = trainData.data[{ {},i,{},{} }]:std()
   trainData.data[{ {},i,{},{} }]:add(-mean[i])
   trainData.data[{ {},i,{},{} }]:div(std[i])
end

-- Normalize test data, using the training means/stds
--for i,channel in ipairs(channels) do
   -- normalize each channel globally:
   --testData.data[{ {},i,{},{} }]:add(-mean[i])
   --testData.data[{ {},i,{},{} }]:div(std[i])
--end
-- Local normalization
print '==> preprocessing data: normalize all three channels locally'

-- Define the normalization neighborhood:
neighborhood = image.gaussian1D(13)

-- Define our local normalization operator (It is an actual nn module, 
-- which could be inserted into a trainable model):
normalization = nn.SpatialContrastiveNormalization(1, neighborhood)

-- Normalize all channels locally:
for c in ipairs(channels) do
   for i = 1,total do
      trainData.data[{ i,{c},{},{} }] = normalization:forward(trainData.data[{ i,{c},{},{} }])
   end
   --for i = 1,325 do
      --testData.data[{ i,{c},{},{} }] = normalization:forward(testData.data[{ i,{c},{},{} }])
   --end
end

print '==> verify statistics'
for i,channel in ipairs(channels) do
   trainMean = trainData.data[{ {},i }]:mean()
   trainStd = trainData.data[{ {},i }]:std()

   --testMean = testData.data[{ {},i }]:mean()
   --testStd = testData.data[{ {},i }]:std()

   print('training data, '..channel..'-channel, mean: ' .. trainMean)
   print('training data, '..channel..'-channel, standard deviation: ' .. trainStd)

   --print('test data, '..channel..'-channel, mean: ' .. testMean)
   --print('test data, '..channel..'-channel, standard deviation: ' .. testStd)
end


==> preprocessing data: colorspace RGB -> YUV	


==> preprocessing data: normalize each feature (channel) globally	


==> preprocessing data: normalize all three channels locally	


==> verify statistics	


training data, y-channel, mean: -0.014755872570939	
training data, y-channel, standard deviation: 0.87619051860649	


training data, u-channel, mean: 0.01716397024165	
training data, u-channel, standard deviation: 0.86160707019696	


training data, v-channel, mean: -0.015114343104192	
training data, v-channel, standard deviation: 0.85675725071268	


## Model

In [17]:
-- define model to train
classes={'bridge', 'building', 'city', 'eiffel_tower','elephant', 'landscape', 'lion', 'monkey', 'people', 'tower', 'water'}
-- define model to train

model = nn.Sequential()
 --stage 1 : mean+std normalization -> filter bank -> squashing -> max pooling
model:add(nn.SpatialConvolutionMM(3,64,5,5))
model:add(nn.ReLU())
model:add(nn.SpatialMaxPooling(2, 2, 2, 2))
-- stage 2 : filter bank -> squashing -> max pooling
model:add(nn.SpatialConvolutionMM(64,64,5,5))
model:add(nn.ReLU())
model:add(nn.SpatialMaxPooling(2, 2, 2, 2))

-- stage 3 : standard 2-layer neural network
model:add(nn.View(64*13*13))
model:add(nn.Linear(64*13*13, #classes))
--model:add(nn.ReLU())
--model:add(nn.Linear(128,#classes))
model:add(nn.LogSoftMax())

print '==> vizualisation du modèle'
print(model:__tostring())
-- retrieve parameters and gradients. this helps us to use the optim package
parameters,gradParameters = model:getParameters()
-- loss function: negative log-likelihood
criterion = nn.ClassNLLCriterion()
print (criterion)
batchSize = 64 -- sets the mini-Batch size
-- this matrix records the current confusion across classes
confusion = optim.ConfusionMatrix(classes)


==> vizualisation du modèle	
nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> output]
  (1): nn.SpatialConvolutionMM(3 -> 64, 5x5)
  (2): nn.ReLU
  (3): nn.SpatialMaxPooling(2x2, 2,2)
  (4): nn.SpatialConvolutionMM(64 -> 64, 5x5)
  (5): nn.ReLU
  (6): nn.SpatialMaxPooling(2x2, 2,2)
  (7): nn.View(10816)
  (8): nn.Linear(10816 -> 11)
  (9): nn.LogSoftMax
}	


nn.ClassNLLCriterion
{
  sizeAverage : true
  output : 0
  gradInput : DoubleTensor - empty
  output_tensor : DoubleTensor - size: 1
  target : LongTensor - size: 1
  total_weight_tensor : DoubleTensor - size: 1
}


In [18]:
-- training function
collectgarbage()
function train(dataset)
   -- epoch tracker
   epoch = epoch or 1

   -- do one epoch
   print('<trainer> on training set:')
   print("<trainer> online epoch # " .. epoch .. ' [batchSize = ' .. batchSize .. ']')
   for t = 1,dataset:size(),batchSize do

      -- create mini batch
      local inputs = {}
      local targets = {}
      for i = t,math.min(t+batchSize-1,dataset:size()) do
         -- load new sample
         local input = dataset.data[i]
         local target = dataset.labels[i]
         table.insert(inputs, input)
         table.insert(targets, target)
      end
      -- create closure to evaluate f(X) and df/dX
      local feval = function(x)
                       -- get new parameters
                       if x ~= parameters then
                          parameters:copy(x)
                       end

                       -- reset gradients
                       gradParameters:zero()

                       -- f is the average of all criterions
                       local f = 0

                       -- evaluate function for complete mini batch
                       for i = 1,#inputs do
                          -- estimate f
                          local output = model:forward(inputs[i])
                          local err = criterion:forward(output, targets[i])
                          f = f + err

                          -- estimate df/dW
                          local df_do = criterion:backward(output, targets[i])
                          model:backward(inputs[i], df_do)

                          -- update confusion
                          confusion:add(output, targets[i])                        
                       end

                       -- normalize gradients and f(X)
                       gradParameters:div(#inputs)
                       f = f/#inputs

                       -- return f and df/dX
                       return f,gradParameters
                    end


      config = config or {learningRate = 1e-3,
              weightDecay = 0,
                momentum = 0,
              learningRateDecay = 5e-7}
      optim.sgd(feval, parameters, config)
   end

   -- print confusion matrix
   print(confusion)
   confusion:zero()

   -- next epoch
   epoch = epoch + 1
end


In [19]:
--classes={'bridge', 'building', 'city', 'eiffel_tower','elephant', 'landscape', 'lion', 'monkey', 'people', 'tower', 'water'}
totaltest = 0
for dir in paths.iterdirs("./dataset/val/") do
    files = {}
    print(files)
    for file in paths.files(paths.concat('./dataset/val/', dir)) do
       if file:find('JPEG' .. '$') then
          table.insert(files, 1)
       end
    end
   
    if #files == 0 then
       error('given directory doesnt contain any files of type: ')
    end
    print(totaltest)
    totaltest = totaltest + #files
    collectgarbage()
end

print('total', totaltest)

imagesAll = torch.Tensor(totaltest,3,64,64) 
labelsAll = torch.Tensor(totaltest)
compteur = 0
NumberOfImages = {0}
for dir in paths.iterdirs("./dataset/val/") do
    compteur = compteur +1
    files = {}
    for file in paths.files(paths.concat('./dataset/val/', dir)) do
       if file:find('JPEG' .. '$') then
          table.insert(files, paths.concat(paths.concat('./dataset/val/',dir), file))
       end
    end
   
    if #files == 0 then
       error('given directory doesnt contain any files of type: ')
    end

    lastElement = NumberOfImages[#NumberOfImages]
    table.insert(NumberOfImages, #files + lastElement)
    for i=1,(#files) do
        temp = image.load(files[i])
        temp2 = torch.Tensor(3,64,64) 
        if (temp:size()[1]==1) then
            temp2[{{1},{},{}}] = temp
            temp2[{{2},{},{}}] = temp
            temp2[{{3},{},{}}] = temp
        else 
            temp2 = temp
        end
        imagesAll[i+lastElement] = temp2 
        labelsAll[i+lastElement] = compteur
    end
    collectgarbage()
end


-- Nombre d'images courleurs: 1683 
testData = {
    data = torch.Tensor(totaltest, 3, 64, 64),
    labels = torch.Tensor(totaltest),
    size = function() return totaltest end,

}


for i=1,totaltest do
   testData.data[i] = imagesAll[i]
   testData.labels[i] = labelsAll[i]
end

-- Convert all images to YUV
print '==> preprocessing data: colorspace RGB -> YUV'
for i = 1,totaltest do
   testData.data[i] = image.rgb2yuv(testData.data[i])
end
--for i = 1,325 do
  -- testData.data[i] = image.rgb2yuv(testData.data[i])
--end
-- Name channels for convenience
channels = {'y','u','v'}

-- Normalize each channel, and store mean/std
-- per channel. These values are important, as they are part of
-- the trainable parameters. At test time, test data will be normalized
-- using these values.
print '==> preprocessing data: normalize each feature (channel) globally'
--mean = {}
--std = {}
for i,channel in ipairs(channels) do
   -- normalize each channel globally:
   testData.data[{ {},i,{},{} }]:add(-mean[i])
   testData.data[{ {},i,{},{} }]:div(std[i])
end

-- Local normalization
print '==> preprocessing data: normalize all three channels locally'

-- Define the normalization neighborhood:
neighborhood = image.gaussian1D(13)

-- Define our local normalization operator (It is an actual nn module, 
-- which could be inserted into a trainable model):
normalization = nn.SpatialContrastiveNormalization(1, neighborhood)

-- Normalize all channels locally:
for c in ipairs(channels) do

   for i = 1,totaltest do
      testData.data[{ i,{c},{},{} }] = normalization:forward(testData.data[{ i,{c},{},{} }])
   end
end

print '==> verify statistics'
for i,channel in ipairs(channels) do

   testMean = testData.data[{ {},i }]:mean()
   testStd = testData.data[{ {},i }]:std()


   print('test data, '..channel..'-channel, mean: ' .. testMean)
   print('test data, '..channel..'-channel, standard deviation: ' .. testStd)
end


{
}


0	


{
}


32	


{
}


...

==> preprocessing data: normalize all three channels locally	


==> verify statistics	


test data, y-channel, mean: -0.014682043068754	
test data, y-channel, standard deviation: 0.8770891870443	


test data, u-channel, mean: 0.018090772041077	
test data, u-channel, standard deviation: 0.86623959227628	


test data, v-channel, mean: -0.016709132318742	
test data, v-channel, standard deviation: 0.86127668997906	


In [20]:
-- test function
function test(dataset)

   -- test over given dataset
   print('<trainer> on testing Set:')
   for t = 1,dataset:size() do
      -- get new sample
      local input = dataset.data[t]
      local target = dataset.labels[t]

      -- test sample
      local pred = model:forward(input)
      confusion:add(pred, target)
   end

   -- print confusion matrix
   print(confusion)
   confusion:zero()

end


In [ ]:
i=0
while i<10 do
train(trainData)
test(testData)
i=i+1
end

<trainer> on training set:	
<trainer> online epoch # 1 [batchSize = 64]	


ConfusionMatrix:
[[     125       0       1      32       0       0       9      15       0       0       0]   68.681% 	[class: bridge]
 [     138      19       0       0       0       0       0       0       0       0       0]   12.102% 	[class: building]
 [       0      85       0       0       0       0       0       0       0       0       0]   0.000% 	[class: city]
 [       0      17       0       0       0       0       0       0       0       0       0]   0.000% 	[class: eiffel_tower]
 [       1     133       1       0      88       0       0       0       0       0       0]   39.462% 	[class: elephant]
 [       0       0       0       0      44       0       0       0       0       0       0]   0.000% 	[class: landscape]
 [       0       0       0       0     163       0     125       0       0       0       0]   43.403% 	[class: lion]
 [       0       0       0       0       0       0     206       9       0       0       0]   4.186% 	[class: monkey]
 [       0       0       0

ConfusionMatrix:
[[       0       0       0       0       0       0       0       0       0       0      32]   0.000% 	[class: bridge]
 [       0       0       0       0       0       0       0       0       0       0      32]   0.000% 	[class: building]
 [       0       0       0       0       0       0       0       0       0       0      32]   0.000% 	[class: city]
 [       0       0       0       0       0       0       0       0       0       0      32]   0.000% 	[class: eiffel_tower]
 [       0       0       0       0       0       0       0       0       0       0      32]   0.000% 	[class: elephant]
 [       0       0       0       0       0       0       0       0       0       0      32]   0.000% 	[class: landscape]
 [       0       0       0       0       0       0       0       0       0       0      32]   0.000% 	[class: lion]
 [       0       0       0       0       0       0       0       0       0       0      32]   0.000% 	[class: monkey]
 [       0       0       0    

ConfusionMatrix:
[[       9       0       0       0       0       0       0       0       0       0     173]   4.945% 	[class: bridge]
 [     148       3       0       0       0       0       0       0       0       0       6]   1.911% 	[class: building]
 [      66      18       0       0       0       0       0       0       0       0       1]   0.000% 	[class: city]
 [      11       6       0       0       0       0       0       0       0       0       0]   0.000% 	[class: eiffel_tower]
 [      87      69       0       0      65       0       0       0       0       0       2]   29.148% 	[class: elephant]
 [       0       0       0       0      44       0       0       0       0       0       0]   0.000% 	[class: landscape]
 [       0       0       0       0     193       0      95       0       0       0       0]   32.986% 	[class: lion]
 [       0       0       0       0       0       0     215       0       0       0       0]   0.000% 	[class: monkey]
 [       0       0       0  

      4 : eiffel_tower
      5 : elephant
      6 : landscape
      7 : lion
      8 : monkey
      9 : people
      10 : tower
      11 : water
    }
  _prediction : FloatTensor - size: 11
  _pred_idx : LongTensor - size: 1
  nclasses : 11
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 11
  totalValid : 0.10535506402794
}
<trainer> on testing Set:	


ConfusionMatrix:
[[       0       0       0       0       0       0       0       0       0       0      32]   0.000% 	[class: bridge]
 [       0       0       0       0       0       0       0       0       0       0      32]   0.000% 	[class: building]
 [       0       0       0       0       0       0       0       1       0       0      31]   0.000% 	[class: city]
 [       0       0       0       0       0       0       0       0       0       0      32]   0.000% 	[class: eiffel_tower]
 [       0       0       0       0       0       0       1       0       0       0      31]   0.000% 	[class: elephant]
 [       0       0       0       0       0       0       1       0       0       0      31]   0.000% 	[class: landscape]
 [       0       0       0       0       0       0       0       0       0       0      32]   0.000% 	[class: lion]
 [       0       0       0       0       0       0       0       0       0       0      32]   0.000% 	[class: monkey]
 [       0       0       0    

 nclasses : 11
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 11
  totalValid : 0
}
<trainer> on training set:	
<trainer> online epoch # 3 [batchSize = 64]	
